In [1]:
# useing this variable for saving models and summaries
exp_name = "DWN_Wmultitask_light_cpd_64"

## Importing Libraries

In [2]:
%load_ext autoreload
%autoreload 2
import tensorflow as tf
import numpy as np
from hdf5storage import savemat
import hickle 

import sys
sys.path.append('../../src')

In [3]:
import models.wavelet_conv_model_light as wavelet_conv_model
import training
import evaluation.nn_eval_multitask as nn_eval

## Loading synthetic data

In [4]:
#loading data
data_path= '../../data/opp8.hkl'
with open(data_path, 'r') as fin:
    data = hickle.load(fin)

In [5]:
train_data = data['train_data']
validation_data = data['validation_data']
test_data = data['test_data']

train_gt = data['train_gt']
validation_gt = data['validation_gt']
test_gt = data['test_gt']

In [6]:
train_data.shape, train_gt.shape

((691, 1024, 77), (691, 1024, 19))

In [7]:
test_data.shape, test_gt.shape

((119, 1024, 77), (119, 1024, 19))

## Model

In [8]:
# Model parameters
n_variables = 77
learning_rate = 0.001
batch_size = 100

wavelet_levels = 5    #number of wavelet layer to have in the network
convolution_levels = 3    #number of convolution to have in the network
pooling_stride = [4, 4, 4]
weights_shape = [[9, n_variables, 32],
                 [5, 32, 32],
                 [5, 32, 32]]
bias_shape = [32, 32, 32]
wavelet_kernel_size = 5    #high_pass and low_pass dimensions
activation = tf.nn.relu
n_classes = 19

##positive weight for weighted cross entropy
s = train_gt.shape
resolution = np.prod(pooling_stride)
pooled_gt = np.reshape(train_gt[:,:(s[0]*s[1]/resolution)*resolution,:], (s[0]*(s[1]/resolution), resolution, s[2])).max(axis=1)
pos_weight = (1.0 - pooled_gt.mean(axis = 0)) / pooled_gt.mean(axis=0)

variables_weights = np.ones((n_classes,))

In [9]:
train_data.shape

(691, 1024, 77)

In [10]:
pos_weight.shape

(19,)

In [11]:
pos_weight

array([  0.16329966,  34.21019108,  34.43589744,  37.12413793,
        36.10067114,  26.70927318,  27.27621483,  39.94814815,
        42.01945525,  46.65517241,  51.89952153,  48.57847534,
        51.1509434 ,  43.40160643,  44.68595041,  33.7672956 ,
        10.65015806,  36.60544218,   2.20649652])

In [12]:
# Launch the graph
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

In [13]:
summary_path = '../../summary/' + exp_name 
checkpoint_path = '../../model/' + exp_name + '/checkpoint' 

In [14]:
model = wavelet_conv_model.Model(n_variables = n_variables,
                                 learning_rate = learning_rate,
                                 n_classes = n_classes)

In [15]:
model.build_neural_wavelet_layer(pos_weight = pos_weight, 
                                 learning_rate = learning_rate,
                                 wavelet_levels = wavelet_levels,    
                                 convolution_levels = convolution_levels,    
                                 pooling_stride = pooling_stride,     
                                 weights_shape = weights_shape,
                                 bias_shape = bias_shape,
                                 wavelet_kernel_size = wavelet_kernel_size,    
                                 activation = activation,
                                 trainable_wavelet=True,
                                 trainable_conv=True,
                                 trainable_last=True,
                                 variables_weights = variables_weights)

In [ ]:
trainer = training.Trainer(model = model,
                    data_train = train_data,
                    ground_truth_train = train_gt,
                    data_validation = test_data,
                    ground_truth_validation = test_gt,
                    sess = sess,
                    summary_path = summary_path,
                    batch_size = batch_size,
                    global_step = 0,
                    eval_type = 'multitask')

In [ ]:
trainer.train(max_iter = 20000,
                train_eval_step = 100, 
                validation_eval_step = 100,
                display_step = 100)

Train Batch Evaluation
ACTIVITY RECOGNITION
AUC = 0.077 vs. 0.078
F1 = 0.126 vs. 0.117
Accuracy = 0.009 vs. 0.056
CHANGE DETECTION
AUC = 0.254 vs. 0.315
F1 = 0.471 vs. 0.473
Precision = 0.312 vs. 0.312
Recall = 0.965 vs. 0.978
Validation
ACTIVITY RECOGNITION
AUC = 0.067 vs. 0.070
F1 = 0.103 vs. 0.107
Accuracy = 0.010 vs. 0.056
CHANGE DETECTION
AUC = 0.136 vs. 0.171
F1 = 0.316 vs. 0.313
Precision = 0.189 vs. 0.185
Recall = 0.954 vs. 0.997
Minibatch Loss= 1.240703


In [ ]:
#Save the model
saver = tf.train.Saver()
saver.save(sess, checkpoint_path, global_step=trainer.global_step)

In [ ]:
#load the model
saver = tf.train.Saver()
#saver.restore(sess, "../model" + "/CNN7002_Gleam" + "/CNN7002_Gleam")
saver.restore(sess, '../../model/DWN_Wmultitask_light_cpd_64/checkpoint-20000')


In [ ]:
#This is the model evaluation on test dataset
result = nn_eval.evaluate_model (model, sess, test_data, test_gt)

In [ ]:
out = nn_eval.deploy(model, sess, test_data, test_gt)

In [ ]:
#shape of probability_change is (batch size, T, num_classes)
#since num_classes = 1 we squeeze that axis
#probability_change = np.squeeze(probability_of_pos, axis = 2)

In [ ]:
savemat('../../poc/poc'+exp_name+'.mat', {'poc':out[:,:,-1]})

In [ ]:
1